In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
#载入数据
mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)

# 输入图片28*28
n_inputs=28 #输入一行 一行有28个数据
max_time=28 #一共28行
lstm_size=100 #隐藏单元
n_classes=10 #十个分类
batch_size=50 #批次五十个样本
n_batch=mnist.train.num_examples//batch_size #计算一共多少批次

x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])

#初始化权值
weights=tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1))
#初始化偏置值
biases=tf.Variable(tf.constant(0.1,shape=[n_classes]))

#定义RNN网络
def RNN(X,weights,biases):
    #inputs=[batch_size,max_time,n_inputs]
    inputs=tf.reshape(X,[-1,max_time,n_inputs])
    #定义LSTM基本CELL
    lstm_cell=tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(lstm_size) 
    #final_state[0] cell state
    #final_state[1] hidden_state
    outputs,final_states=tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    results=tf.nn.softmax(tf.matmul(final_states[1],weights)+biases)
    return results

#计算RNN返回结果
prediction=RNN(x,weights,biases)

#损失函数
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))

#使用adamoptimizer优化
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#结果存放在布尔型列表中  定义准确率 比较两个agrmax大小，相同就是TRUE 不同就是FALSE。 argmax求最大值所在位置
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))

#求准确率 (tf.cast()将布尔型数据转化成数值型 然后计算准确率)
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
            
        acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print ("Iter"+str(epoch)+", testing accuracy="+str(acc))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter0, testing accuracy=0.7387
Iter1, testing accuracy=0.8508
Iter2, testing accuracy=0.8819
Iter3, testing accuracy=0.9023
Iter4, testing accuracy=0.9161
Iter5, testing accuracy=0.9223
